<a href="https://colab.research.google.com/github/arriyan-aa/AI-Hand-Computer-Vision/blob/model-setup/cv_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><h1> <b>AI-Hand-Computer-Vision </b> </center> </h1>

Distinguishing between AI-generated hands vs. real hands. Our project aims to address the issue of AI in art and other media. This is an important problem because the use of AI generated material in art and media is becoming increasingly more common and has brought up discussions of ethics and copyright. This model would help distinguish real human hands versus AI generated ones.

# Libraries

In [ ]:
!pip install pandas
!pip install matplotlib
!pip install numpy

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Setup

Pip install `ultralytics` and [dependencies](https://github.com/ultralytics/ultralytics/blob/main/requirements.txt) and check software and hardware.

In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
!pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics 8.3.220 🚀 Python-3.12.12 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 39.7/107.7 GB disk)


YOLOv8 may be used directly in the Command Line Interface (CLI) with a `yolo` command for a variety of tasks and modes and accepts additional arguments, i.e. `imgsz=640`. See a full list of available `yolo` [arguments](https://docs.ultralytics.com/usage/cfg/) and other details in the [YOLOv8 Predict Docs](https://docs.ultralytics.com/modes/train/).


if CLI the format should be:

    yolo TASK MODE ARGS

  Where:

    TASK (optional) is one of (detect, segment, classify, pose)


    MODE (required) is one of (train, val, predict, export, track)


    ARGS (optional) are arg=value pairs like imgsz=640 that override defaults.


Default ARG values are defined on this page from the cfg/defaults.yaml file.


In [23]:
%matplotlib inline
from PIL import Image

from ultralytics import YOLO
model = YOLO("yolo11n-cls.pt")

# Training

Train YOLOv8 on Detect, Segment, Classify and Pose datasets. See YOLOv8 Train Docs for more information.

In [26]:
#@title Select YOLOv8 🚀 logger {run: 'auto'}
logger = 'Comet' #@param ['Comet', 'TensorBoard']

if logger == 'Comet':
  %pip install -q comet_ml
  import comet_ml; comet_ml.init()
elif logger == 'TensorBoard':
  %load_ext tensorboard
  %tensorboard --logdir .

COMET WARNING: comet_ml.init() is deprecated and will be removed soon. Please use comet_ml.login()


In YOLO (You Only Look Once), a YAML file is used for configuration and setup. It specifies parameters such as paths to datasets, model architecture, training hyperparameters, and class names. The YAML file is essential for defining how the YOLO model should be trained and what it should detect, making it an integral part of customizing the YOLO model for specific object detection tasks.

In [24]:
# Training YOLOv11 on hand dataset for 3 epochs
!yolo classify train model=yolo11n-cls.pt data=drive/MyDrive/hand-dataset epochs=3 imgsz=640


Ultralytics 8.3.220 🚀 Python-3.12.12 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=drive/MyDrive/hand-dataset, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=3, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train8, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, po